<h1 id="tocheading">Table of Contents and Notebook Setup</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.get('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

# Some Math Prereqs

The <b> covariance </b> of two distributions X and Y is defined as

$$cov(X,Y) = \frac{1}{n-1}\sum_{i=1}^n (x_i-\mu_x)(y_i-\mu_y) $$

i represents the 'i'th measurement 

n is the total number of measurements 

$x_i$ and $y_i$ are individual measurements

$\mu_x$ and $\mu_y$ are the mean values of X and Y

This gives us information about how the two variables deviate from their expected values (means) and if they do it at the same time. If they both deviate positively or negatively at the same time, then we get a large contribution from the sum. If one deviates positively and the other negatively then we get a large negative number. Big positive numbers mean the variables are correlated (big negative also means they're related in some way too- as one goes up the other goes down).

The problem is that these large or small numbers depend on the scale of the units we use for measurement. We want a quantity that we know is the same for all distributions. We can divide by their <b> variances </b> (related to standard deviation).

$$\sigma_x \equiv cov(X,X) = \frac{1}{n-1}\sum_{i=1}^n (x_i-\mu_x)^2 $$

We define the <b> correlation </b> (or more precisely: the <i> linear correlation </i>) as follows:

$$corr(X,Y)=\frac{cov(X,Y)}{\sqrt{\sigma_x \sigma_y}}$$

The inequaltity $-1 \leq corr(X,Y) \leq 1$ always holds. This can be shown through the Cauchy-Schwartz inequality (quantities $x_i-\mu_x$ are elements of a vector).

If $corr(X,Y)=1$ then the values are perfectly correlated (in the vector space of measurements $x_i-\mu_x$ and $y_i-\mu_y$ they point in the same direction). If $corr(X,Y)=-1$ then the values are perfectly uncorrelated (in the vector space they point in opposite direction).

Such information is when comparing stock trends. Suppose that amazon always drops the day after microsoft goes up and we find a strict anticorrelation ($corr(X,Y)=-1$). In the future, when microsoft goes up, we may want to sell our amazon stock as we know its going to drop- then pick it up the next day for a discount.

# Basic Mathematical Functions of Pandas

pandas objects are equipt to deal with a variety of mathematical and statistical functions, and can also deal with missing data.

In [3]:
df = pd.DataFrame([[1.4,np.nan],[7.1,-4.5],[np.nan,np.nan],[0.75,-1.3]],
                  index=['a','b','c','d'], columns=['one','two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


## The Sum Method

Use <b> sum </b> to return the column sums. 

In [4]:
df.sum()

one    9.25
two   -5.80
dtype: float64

We can also use axis='columns' to sum <i> across </i> the columns instead.

In [5]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

## The Mean Method

We can exclude rows with NA values if we like:

In [6]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

## Accumulation Method (Integration of Rows/Columns)

In [7]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


We can use this for integration if we like.

In [8]:
dx = 0.001
(df*dx).cumsum()

,one,two
a,0.00140,NaN
b,0.00850,-0.0045
c,NaN,NaN
d,0.00925,-0.0058


## Basic Statistical Method 

We can use the describe method to learn about the rows and columns.

In [9]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


## Summary

See table 5-8 on page 160 of textbook for all simple methods.

# Correlation and Covariance

## Introduction to Correlation in DataFrames

Correlation and Covariance look at the relationship between two data sets. Below we compare stock datasets. 

In [10]:
import pandas_datareader.data as web
stocks = ['AMZN', 'GOOG', 'AAPL', 'TD', 'JNJ', 'IBM']

start = pd.datetime(2017, 7, 29)
end = pd.datetime(2018, 8, 2)
f1 = web.DataReader(stocks, 'iex', start, end)
f1['open'].head() #opening price for the stock on that day

ImportError: cannot import name 'is_list_like'

Lets apply some functions and see how the stock changes at the beginning and the end of the day. Recall that functions like the one below can operate on rows or columns of dataframes; in this case we choose column headers.

In [ ]:
def find_change(x, stock):
    return x['close'][stock]-x['open'][stock]

stock_day_changes = pd.DataFrame([f1.apply(find_change, axis='columns', args=(stock,)) 
                                  for stock in stocks], index=stocks)
stock_day_changes = stock_day_changes.transpose()
stock_day_changes.head()

Correlation for entire DataFrame:

In [ ]:
stock_day_changes.corr()

Or we can just select single elements:

In [ ]:
stock_day_changes['AMZN'].corr(stock_day_changes['GOOG'])

Or we can select rows:

In [ ]:
stock_day_changes.corrwith(stock_day_changes.AMZN)

# Unique Values, Counting Occurences, and Membership of Elements in a Series

The pandas module has even more methods for determining if elements in a series are unique. This is obviously useful for DataFrames as the rows and columns can be extracted as Series.

## Uniqueness

In [ ]:
obj = pd.Series(['c', 'a', 'c', 'b', 'a', 'c', 'b', 'a', 'c'])
uniques = obj.unique()
uniques

## Counting Occurences

In [ ]:
obj = pd.Series(['c', 'a', 'c', 'b', 'a', 'c', 'b', 'a', 'c'])
obj.value_counts()

The returned Series is sorted by the number of occurences. We can choose not to have this as well:

In [ ]:
pd.value_counts(obj.values, sort=False)

## Membership

Sometimes we want to see if an element is contained in a Series. We can use the <i> isin </i> method for this.

In [ ]:
mask = obj.isin(['b', 'c'])
mask

Then we can use the mask to extract the elements that we want.

In [ ]:
obj[mask]

This can often make boolean indexing DataFrames easier when one has lots of conditions.

Suppose we have a Series of distinct values and a Series of non-distinct values like below:

In [ ]:
to_match = pd.Series(['c','b','c','a','b'])
unique_vals = pd.Series(['b','a','c'])

We can use the <i> Index.get_indexer </i>  method to give an index array from the unique values:

In [ ]:
pd.Index(unique_vals).get_indexer(to_match)